In [1]:
import sys
import os
sys.path.append("../../libraries/pvae/")
from pvae.manifolds.poincareball import PoincareBall
from pvae.manifolds.euclidean import Euclidean
from pvae.models.architectures import EncWrapped, DecWrapped
from pvae.distributions.wrapped_normal import WrappedNormal
from pvae.distributions.riemannian_normal import RiemannianNormal
from pvae.ops.manifold_layers import GeodesicLayer
from pvae.objectives import vae_objective
from torch.distributions.normal import Normal
sys.path.append("../../libraries/")
from HypHC.optim.radam import RAdam
from HypHC.utils.poincare import project
from HypHC.utils.visualization import plot_tree_from_leaves
from HypHC.utils.linkage import nn_merge_uf_fast_np, sl_from_embeddings
from HypHC.utils.metrics import dasgupta_cost
sys.path.append("../hyperLAI")
import math
import torch
import joblib
from torch import nn
import networkx as nx
import sknetwork
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
import scipy.cluster.hierarchy as sch
from models.hyperbolic_hc_loss import HyperbolicHCLoss
from models.encoder_decoder_architectures import *
from models.vae_model import vae_model
from torch.utils.data import SubsetRandomSampler, DataLoader, Subset
from torch.optim import Adam
from utils.sim_funcs import sim_func_dict
from utils.model_utils import *
from features.hyperLAIdataset import HyperLoader
from models.fc_model import fc_model

enc_dec_dict = {"fc_wrapped_encoder": fc_wrapped_encoder, "fc_wrapped_decoder": fc_wrapped_decoder, "fc_geodesic_decoder": fc_geodesic_decoder,
               "fc_wrapped_decoder_rawvals": fc_wrapped_decoder_rawvals} 
manifold_dict = {"PoincareBall": PoincareBall, "Euclidean": Euclidean}
distribution_dict = {"WrappedNormal": WrappedNormal, "Normal": Normal, "RiemannianNormal": RiemannianNormal}


# Load Model and Obtain Embeddings

In [2]:
model_dir = "/scratch/users/patelas/hyperLAI/models/vae_models/full_pop_models/500000_threelayers_tempnegsix_shuffle_newearlystop/"
args = read_config(model_dir + "vae_config.json")
indices = np.load(args["index_loc"] + "valid_indices.npy")
# indices = np.load("/scratch/users/patelas/hyperLAI/ancestry_training_splits/80_10_10/test_indices.npy")

In [3]:
#Load validation dataset
eval_dataset = HyperLoader(args["data_dir"], indices, args["restrict_labels"], args["chromosome"])
eval_loader = DataLoader(eval_dataset, batch_size=64)



In [4]:
# pca_model = joblib.load(args["output_dir"] + "pca_model.joblib")
# eval_snps_old = eval_dataset.snps.copy()
# eval_dataset.snps = pca_model.transform(eval_dataset.snps)

In [5]:
#Define manifold and model
manifold = manifold_dict[args["manifold"]](args["embedding_size"])
enc_type, dec_type  = enc_dec_dict[args["enc_type"]], enc_dec_dict[args["dec_type"]]
encoder = enc_type(manifold, eval_dataset[0][0].shape[-1], args["num_encoder_int_layers"], 
                   args["encoder_int_layer_sizes"], args["encoder_dropout_vals"], args["embedding_size"])
decoder = dec_type(manifold, eval_dataset[0][0].shape[-1], args["num_decoder_int_layers"], 
                   args["decoder_int_layer_sizes"], args["decoder_dropout_vals"], args["embedding_size"])
model = vae_model(encoder, decoder, manifold, distribution_dict[args["posterior_dist"]],
                  distribution_dict[args["prior_dist"]], args["prior_mean"], args["prior_std"], args["temperature"], args["init_size"], args["min_scale"], args["max_scale"])


In [6]:
model_info = torch.load("%smodel.pt"%(model_dir))
model.load_state_dict(model_info["model_state"])

<All keys matched successfully>

In [7]:
# torch.save(model.state_dict(), "/home/users/patelas/hyperLAI/pretrained_models/vae_model.pt")

OSError: [Errno 28] No space left on device

In [ ]:
#Run model on validation data
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)
model.eval()
snps, embeddings, recons, suppop_labels, pop_labels = [], [], [], [], []
with torch.no_grad():
    for i, (snp_data, suppop, pop) in enumerate(eval_loader):
        embs = model.embed(snp_data.float().to(device))
        recon = model.generate(embs.to(device))
        snps.append(snp_data.cpu())
        embeddings.append(embs.cpu())
        recons.append(recon.cpu())
        suppop_labels.append(suppop)
        pop_labels.append(pop)

In [ ]:
#Convert to numpy arrays
snps = torch.cat(snps).numpy()
embeddings = torch.cat(embeddings).numpy()
recons = torch.cat(recons).numpy()
suppop_labels = torch.cat(suppop_labels).numpy()
pop_labels = torch.cat(pop_labels).numpy()

# Plot Node Embeddings (Perhaps as PCA)

In [ ]:
def plot_weights_pca(embeddings, labels, annotations=None):
    '''
    Plot embeddings. Uncomment/comment as necessary depending on if you want raw embeddings or PCA
    '''
#     weights_pca = PCA().fit_transform(embeddings)
#     scplot = sns.scatterplot(x=weights_pca[:,0], y=weights_pca[:,1], hue=labels)
    label_dict = {"EUR": "Europe", "EAS": "East Asia", "AMR": "Americas", "AFR": "Africa","SAS": "South Asia", 
                  "OCE": "Oceania", "WAS": "West Asia"}
#     h_order = ["EUR", "EAS", "AMR", "AFR", "SAS", "OCE", "WAS"]
    h_order = ["Europe", "East Asia", "Americas", "Africa", "South Asia", "Oceania", "West Asia"]
    sns.set_style('white')
    scplot = sns.scatterplot(x=embeddings[:,0], y=embeddings[:,1], hue=[label_dict[x] for x in labels], hue_order=h_order)
#     plt.xlabel("PC1")
#     plt.ylabel("PC2")
#     plt.title("PCA of Embedding Weights")
    plt.xlabel("Embedding 1", fontsize=20)
    plt.ylabel("Embedding 2", fontsize=20)
    plt.title("Embedding Weights - VAE Model", fontsize=20)
    plt.legend(fontsize=20)
    if annotations is not None:
        for line in range(len(labels)):
#             if weights_pca[line,1] > -0.3:
#             if annotations[line] != "Mozabite":
#                 continue
            scplot.text(embeddings[line,0]+0.001, embeddings[line,1], 
                         annotations[line], horizontalalignment='left', color='black', size=8)


In [ ]:
plt.figure(figsize=[15, 15], dpi=300)
sp_full = [eval_dataset.suppop_label_index[x] for x in suppop_labels]
p_full = [eval_dataset.pop_label_index[x] for x in pop_labels]
plot_weights_pca(embeddings, sp_full, annotations=None)
plt.show()

# Produce, Plot, and Evaluate Tree

In [ ]:
def decode_tree(model, embeddings, device, fast_decoding):
    """Build a binary tree (nx graph) from leaves' embeddings. Assume points are normalized to same radius.
        Taken from HypHC repo (https://github.com/HazyResearch/HypHC)
    
    """
    with torch.no_grad():
        leaves_embeddings = model.HypHCLoss.normalize_embeddings(torch.tensor(embeddings).to(device))
        leaves_embeddings = project(leaves_embeddings).cpu()
    sim_fn = lambda x, y: torch.sum(x * y, dim=-1)
    if fast_decoding:
        parents = nn_merge_uf_fast_np(leaves_embeddings, S=sim_fn, partition_ratio=1.2)
    else:
        parents = sl_from_embeddings(leaves_embeddings, sim_fn)
    tree = nx.DiGraph()
    for i, j in enumerate(parents[:-1]):
        tree.add_edge(j, i)
    return tree


In [ ]:
#Plot tree
embs_tree = embeddings
tree = decode_tree(model, embs_tree, device, True)
with torch.no_grad():
    embs_normed = model.HypHCLoss.normalize_embeddings(torch.tensor(embs_tree).to(device))
    leaves_embeddings = project(embs_normed).cpu().numpy()
fig = plt.figure(figsize=(15, 15), dpi=300)
ax = fig.add_subplot(111)
ax = plot_tree_from_leaves(ax, tree, leaves_embeddings * 100, labels=sp_full)

# Uncomment to add subpopulation labels - but it's very crowded
# to_write = [np.random.choice([True, False], p=[0.1, 0.9]) for x in range(len(suppop_labels))]
# for line in range(len(suppop_labels)):
#     if to_write[line]:
#         ax.text(leaves_embeddings[line,0] * 100 +0.001, leaves_embeddings[line,1] * 100, 
#                      p_full[line], ha='left', 
#                 color='black', va="baseline", rotation=0, size=6)

plt.show()


In [ ]:
def make_pairwise_similarities(data, sim_func):
    '''
    Creates a similarity matrix from the SNP data using the specified similarity function
    This is a numpy version of the function used in training
    '''
    sim_matrix = np.zeros((data.shape[0], data.shape[0]))
    #Fill in matrix
    for ind in range(data.shape[0]):
        for ind2 in range(data.shape[0]):
            sim_matrix[ind][ind2] = sim_func(data[ind], data[ind2])
    #Divide by maximum for normalization
    sim_matrix /= np.amax(sim_matrix)
    return sim_matrix



def make_pairwise_label_similarities(data):
    sim_matrix = np.zeros((data.shape[0], data.shape[0]))
    #Fill in matrix
    for ind in range(data.shape[0]):
        for ind2 in range(data.shape[0]):
            sim_matrix[ind][ind2] = (int(p_full[ind] == p_full[ind2]) + int(sp_full[ind] == sp_full[ind2])) // 2
    #Divide by maximum for normalization
    sim_matrix /= np.amax(sim_matrix)
    return sim_matrix



In [ ]:
sim_mat = make_pairwise_similarities(eval_dataset.snps, lambda x,y: (np.sum(x==y)) / len(x))
# sim_mat = make_pairwise_similarities(eval_snps_old, lambda x,y: (np.sum(x==y)) / len(x))

# sim_mat = make_pairwise_label_similarities(eval_dataset.snps)
# train_sim_mat = np.load("/scratch/users/patelas/hyperLAI/snp_data/whole_genome/variance_filtered_500000_updated/sim_mats/train_hamming.npy")

In [ ]:
# sim_mat_minmax = (sim_mat - 0.451) / 0.549
# sim_mat_minmax = np.where(sim_mat_minmax >= 0, sim_mat_minmax, 0)

In [ ]:
# np.min(sim_mat_minmax)

In [ ]:
#Calculates Dasgupta's cost of the tree
print(dasgupta_cost(tree, sim_mat))

In [ ]:
# np.save("/scratch/users/patelas/hyperLAI/snp_data/whole_genome/variance_filtered_500000_updated/sim_mats/train_hamming.npy", sim_mat)





# Test Reconstruction Ability

This part does a few different things, all relating to the differences between true and reconstructed data. It also runs PCA and inverse PCA for comparison purposes

In [ ]:
#Some basic stats of the differences between true and reconstructed data
abs_diffs = np.absolute(snps-recons)
print(np.min(abs_diffs), np.mean(abs_diffs), np.max(abs_diffs))

In [ ]:
#Reconstruction accuracy
preds = recons > 0.5
abs_preds = np.sum(snps == preds) / len(snps.flatten())
print(abs_preds)

In [ ]:
plt.plot(sorted(np.mean(np.absolute(snps-recons),axis=1)))
plt.show()
plt.plot(sorted(np.min(np.absolute(snps-recons),axis=1)))
plt.show()
plt.plot(sorted(np.max(np.absolute(snps-recons),axis=1)))
plt.show()

In [ ]:
#Compare reconstruction performance to reconstruction performance of PCA
recon_pca = PCA(n_components=100)
recon_pca.fit(dataset.snps[train_indices])
comps = recon_pca.transform(snps)
pca_reprod = recon_pca.inverse_transform(comps)
reprod_preds = pca_reprod > 0.5
print(np.sum(snps == reprod_preds) / len(snps.flatten()))

# Test Fidelity of Reproduced Embeddings

This part takes synthetic embeddings (designed to reflect new "samples") and then runs them through the decoder and then the encoder. We then observe how the new embeddings reflect the original embedding space 

In [ ]:
def decoder_acc_test(model, embeddings):
    '''
    Runs reconstruction and then re-embeds 
    '''
    model.eval()
    with torch.no_grad():
        recons = model.generate(embeddings.to(device))
        new_embs = model.embed(recons)
    return new_embs.cpu()

In [ ]:
#We produce "fake embeddings" by adding random noise to current embeddings
fake_embs = embeddings + np.random.normal(0, 0.05, size=embeddings.shape)
#We then run them through the decoder and encoder
test_emb = torch.tensor(fake_embs).float()
recon_embs = decoder_acc_test(model, test_emb).numpy()

In [ ]:
#Take a look at some summary stats 
euc_dists = np.sqrt(np.sum((recon_embs - fake_embs)**2, axis=1))
print(np.min(euc_dists), np.mean(euc_dists), np.max(euc_dists))
axis_dists = np.absolute(recon_embs - fake_embs)


In [ ]:
#Look at pattern of new embeddings and how it compares to the true pattern
plot_weights_pca(recon_embs, sp_full)
plt.show()

# Test Quality of Reconstructions

This part reconstructs data from embeddings, and then finds the minimum L1 distance between each reconstructed genotype and any true genotype. The embeddings provided can be either real or synthetic. 

In [ ]:
def produce_recons(model, embeddings):
    model.eval()
    with torch.no_grad():
        recons = model.generate(torch.tensor(embeddings).float().to(device))
    return recons.cpu()

In [ ]:
def l1_norm_diff(recon, real_snps):
    diffs = np.sum(np.abs(real_snps - (recon > 0.5).astype(float)), axis=1)
    return np.min(diffs)

In [ ]:
rows_to_use = np.random.choice(len(embeddings), 20, replace=False)
fake_embs = embeddings[rows_to_use] + np.random.normal(0, 0.05, size=embeddings[rows_to_use].shape)
# fake_embs = embeddings[rows_to_use]
recons = produce_recons(model, fake_embs).numpy()
min_diffs = np.apply_along_axis(lambda x: l1_norm_diff(x, snps), 1, recons)

In [ ]:
min_diffs

# Dasgupta's Cost Using Regular Hierarchical Clustering

In [ ]:
#Test set Dasgupta's costs
#Single: 68384691.04029198
#Complete: 68183619.215324
#Average: 68071384.105212
#Weighted: 68074478.67049602


In [ ]:
hamming_dist = lambda x, y: (np.sum(x != y) / len(x)) #Same as "hamming" in scipy linkage

In [ ]:
def perform_hc(data, metric, method):
    hc_tree = nx.DiGraph()
    linkage_list = sch.linkage(data, metric=metric, method=method)
    for snp in range(len(data)):
        hc_tree.add_node(snp)
    for ind, extra_clust in enumerate(linkage_list):
        hc_tree.add_node(len(data) + ind)
        hc_tree.add_edge(len(data) + ind, int(extra_clust[0]))
        hc_tree.add_edge(len(data) + ind, int(extra_clust[1]))
    return hc_tree

In [ ]:
hc_tree = perform_hc(eval_dataset.snps, "hamming", "complete")

In [ ]:
hc_tree, tree

In [ ]:
# print(dasgupta_cost(hc_tree, sim_mat), dasgupta_cost(tree, sim_mat))
print(dasgupta_cost(hc_tree, sim_mat))

In [ ]:
test_array = np.array([[1,1,1,1], [1,1,1,0], [0,0,0,0], [1,0,0,0]])
test_tree = perform_hc(test_array, "hamming", "average")

In [ ]:
test_tree.edges

In [ ]:
assert type(tree) == type(hc_tree)

In [ ]:
type(hc_tree)

In [ ]:
#Test that "tree" does not have any true nodes as the start of edges
tree_edges_start = [x[0] for x in tree.edges]
assert min(tree_edges_start) == len(eval_dataset)

In [ ]:
#Test that "hc_tree" does not have any true nodes as the start of edges
hc_tree_edges_start = [x[0] for x in hc_tree.edges]
assert min(hc_tree_edges_start) == len(eval_dataset)

In [ ]:
#Test that both trees have the same number of edges
assert len(tree.edges) == len(hc_tree.edges)

In [ ]:
#Assert that all true nodes are found as the recipient of edges in both trees
tree_edges_end = [x[1] for x in tree.edges]
tree_all_nodes = [x not in tree_edges_end for x in range(len(eval_dataset))]
assert sum(tree_all_nodes) == 0
hc_tree_edges_end = [x[1] for x in hc_tree.edges]
hc_tree_all_nodes = [x not in hc_tree_edges_end for x in range(len(eval_dataset))]
assert sum(hc_tree_all_nodes) == 0


In [ ]:
#Assert that no more than two edges start from the same point
tree_edges_start = [x[0] for x in tree.edges]
tree_counts = [tree_edges_start.count(x) for x in tree_edges_start]
assert max(tree_counts) <= 2
hc_tree_edges_start = [x[0] for x in hc_tree.edges]
hc_tree_counts = [hc_tree_edges_start.count(x) for x in hc_tree_edges_start]
assert max(hc_tree_counts) <= 2


In [ ]:
len(eval_dataset)

# Get Cross-Prediction Statistics

In [ ]:
dcosts = np.array([68103647.032476,
68091529.558308,
68089696.80722,
68092794.567008,
68192833.336424,
68093968.794136,
68165172.798208,
68197330.18846399,
68091692.73715201,
68198013.69252])

In [ ]:
np.mean(dcosts)

In [ ]:
np.std(dcosts) * 1.96 * 1e-7